## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [1]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [33]:
# CodeGrade step1
# Replace None with your code
# Return a table of in officeCode '2' renamed as Boston
def get_boston_employees_column_names():
    query = """
    SELECT e.firstName AS first_name, e.lastName AS last_name, e.jobTitle AS job_title
    FROM employees e
    JOIN offices o ON e.officeCode = o.officeCode
    WHERE o.city = 'Boston';
    """
    df = pd.read_sql(query, conn)
    return df.columns.tolist()
pd.read_sql("SELECT * FROM employees WHERE officeCode = '2'", conn)     
def get_boston_employees():
    query = """
    SELECT e.firstName, e.lastName, e.jobTitle, o.city
    FROM employees e
    JOIN offices o ON e.officeCode = o.officeCode
    WHERE o.city = 'Boston';
    """
    df = pd.read_sql(query, conn)
    return df

# Call the function to execute the query
df_boston_employees = get_boston_employees()
pd.read_sql ("SELECT * FROM employees WHERE officeCode = '2'", conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [ ]:
# CodeGrade step2
# Replace None with your code
# List of Offices and the count of number of employees in each office
def get_office_employee_counts():
    query = """
    SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS employee_count
    FROM offices o
    LEFT JOIN employees e ON o.officeCode = e.officeCode
    GROUP BY o.officeCode, o.city;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_office_employee_counts = get_office_employee_counts()
pd.read_sql("SELECT * FROM offices", conn)

#display the result
print(df_office_employee_counts)

  officeCode           city  employee_count
0          1  San Francisco               6
1          2         Boston               2
2          3            NYC               2
3          4          Paris               5
4          5          Tokyo               2
5          6         Sydney               4
6          7         London               2


## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [ ]:
# CodeGrade step3
# Replace None with your code
# List of employees and location they work 
def get_employee_office_info():
    query = """
    SELECT e.firstName, e.lastName, o.city, o.state
    FROM employees e
    JOIN offices o ON e.officeCode = o.officeCode
    ORDER BY e.firstName, e.lastName;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_employee_office_info = get_employee_office_info()
#display the result
print(df_employee_office_info)


   firstName   lastName           city       state
0       Andy     Fixter         Sydney            
1    Anthony        Bow  San Francisco          CA
2      Barry      Jones         London            
3      Diane     Murphy  San Francisco          CA
4   Foon Yue      Tseng            NYC          NY
5     George     Vanauf            NYC          NY
6     Gerard     Bondur          Paris            
7     Gerard  Hernandez          Paris            
8       Jeff   Firrelli  San Francisco          CA
9      Julie   Firrelli         Boston          MA
10     Larry       Bott         London            
11    Leslie   Jennings  San Francisco          CA
12    Leslie   Thompson  San Francisco          CA
13      Loui     Bondur          Paris            
14      Mami      Nishi          Tokyo  Chiyoda-Ku
15    Martin     Gerard          Paris            
16      Mary  Patterson  San Francisco          CA
17    Pamela   Castillo          Paris            
18     Peter      Marsh        

### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [ ]:
# CodeGrade step4
# Replace None with your code
# List of customers, phone, and salesRepEmployeeNumber with zero orders
def get_customers_with_no_orders():
    query = """
    SELECT c.contactFirstName, c.contactLastName, c.phone, c.salesRepEmployeeNumber
    FROM customers c
    LEFT JOIN orders o ON c.customerNumber = o.customerNumber
    WHERE o.orderNumber IS NULL
    ORDER BY c.contactLastName;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_customers_with_no_orders = get_customers_with_no_orders()
#display the result
print(df_customers_with_no_orders)


   contactFirstName  contactLastName             phone salesRepEmployeeNumber
0            Raanan      Altagar,G M  + 972 9 959 8555                       
1               Mel         Andersen       030-0074555                       
2            Carmen            Anton    +34 913 728555                       
3        Alejandra            Camino     (91) 745 6555                       
4           Philip            Cramer        0555-09555                       
5        Alexander             Feuer       0342-555176                       
6             Keith           Franco        2035557845                   1286
7            Peter           Franken       089-0877555                       
8                Ed         Harrison  +41 26 425 50 01                       
9             Karin          Josephs       0251-555259                       
10           Horst             Kloss       0372-555188                       
11           Armand            Kuger   +27 21 550 3555          

## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [ ]:
# CodeGrade step5
# Replace None with your code
# List of payments and customers with the payment amount and date
def get_payments_with_customers():
    query = """
    SELECT p.customerNumber, c.contactFirstName, c.contactLastName, p.amount, p.paymentDate
    FROM payments p
    JOIN customers c ON p.customerNumber = c.customerNumber
    ORDER BY amount DESC;
    """
    df_payment = pd.read_sql(query, conn)
    return df_payment
# Call the function to execute the query
# display the result
print(get_payments_with_customers())

    customerNumber contactFirstName contactLastName     amount paymentDate
0              462          Violeta         Benitez    9977.85  2003-11-08
1              333              Ben        Calaghan    9821.32  2003-10-17
2              198           Leslie          Taylor    9658.74  2004-12-06
3              471             Sean        Clenahan    9415.13  2004-07-28
4              452          Roland           Mendel    8807.12  2005-05-03
..             ...              ...             ...        ...         ...
268            124            Susan          Nelson   11044.30  2003-04-11
269            148             Eric       Natividad  105743.00  2003-12-26
270            128           Roland          Keitel   10549.01  2003-01-28
271            363          Dorothy           Young   10223.83  2003-01-16
272            124            Susan          Nelson  101244.59  2005-03-05

[273 rows x 5 columns]


## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [ ]:
# CodeGrade step6
# Replace None with your code
# List of employeeNumber, and name with number of customers whose average credit limit is greater than 90,000
def get_employees_with_high_credit_customers():
    query = """
    SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber) AS customer_count
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    WHERE c.creditLimit > 90000
    GROUP BY e.employeeNumber, e.firstName, e.lastName
    ORDER BY customer_count DESC;
    """
    df_credit = pd.read_sql(query, conn)
    return df_credit
# Call the function to execute the query
#display the result
print(get_employees_with_high_credit_customers())

   employeeNumber firstName   lastName  customer_count
0            1401    Pamela   Castillo              10
1            1504     Barry      Jones               9
2            1323    George     Vanauf               8
3            1501     Larry       Bott               8
4            1286  Foon Yue      Tseng               7
5            1370    Gerard  Hernandez               7
6            1165    Leslie   Jennings               6
7            1166    Leslie   Thompson               6
8            1188     Julie   Firrelli               6
9            1216     Steve  Patterson               6
10           1337      Loui     Bondur               6
11           1702    Martin     Gerard               6
12           1611      Andy     Fixter               5
13           1612     Peter      Marsh               5
14           1621      Mami      Nishi               5


### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [ ]:
# CodeGrade step7
# Replace None with your code
# List of product, number of orders for each product 
df_product_sold = pd.read_sql("""
SELECT p.productName, COUNT(o.orderNumber) AS numorders, SUM(od.quantityOrdered) AS totalunits
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
GROUP BY p.productName
ORDER BY totalunits DESC;
""", conn)
def get_top_selling_products():
    return df_product_sold
# Call the function to execute the query
df_top_selling_products = get_top_selling_products()
#display the result
print(df_top_selling_products)

                                 productName  numorders  totalunits
0                1992 Ferrari 360 Spider red         53        1808
1                       1937 Lincoln Berline         28        1111
2                  American Airlines: MD-11S         28        1085
3    1941 Chevrolet Special Deluxe Cabriolet         28        1076
4               1930 Buick Marquette Phaeton         28        1074
..                                       ...        ...         ...
104             1999 Indy 500 Monte Carlo SS         25         855
105                       1911 Ford Town Car         25         832
106         1936 Mercedes Benz 500k Roadster         25         824
107               1970 Chevy Chevelle SS 454         25         803
108                    1957 Ford Thunderbird         24         767

[109 rows x 3 columns]


## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [ ]:
# CodeGrade step8
# Replace None with your code
# List of product name, code, and total number of customers who have ordered 
def get_product_purchasers():
    query = """
    SELECT p.productName, p.productCode, COUNT(DISTINCT o.customerNumber) AS numpurchasers
    FROM products p
    JOIN orderdetails od ON p.productCode = od.productCode
    JOIN orders o ON od.orderNumber = o.orderNumber
    GROUP BY p.productName, p.productCode
    ORDER BY numpurchasers DESC;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_product_purchasers = get_product_purchasers()
#display the result
print(df_product_purchasers)

                             productName productCode  numpurchasers
0            1992 Ferrari 360 Spider red    S18_3232             40
1                     1934 Ford V8 Coupe    S18_2957             27
2               1952 Alpine Renault 1300    S10_1949             27
3                    1972 Alfa Romeo GTA    S10_4757             27
4                       Boeing X-32A JSF    S72_1253             27
..                                   ...         ...            ...
104  1958 Chevy Corvette Limited Edition    S24_2840             19
105                   1949 Jaguar XK 120    S24_2766             18
106                    1952 Citroen-15CV    S24_2887             18
107            1969 Chevrolet Camaro Z28    S24_3191             18
108                  2002 Chevy Corvette    S24_3432             18

[109 rows x 3 columns]


### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [ ]:
# CodeGrade step9
# Replace None with your code
# List of how many customers there are per office 
def get_customers_per_office():
    query = """
    SELECT o.officeCode, o.city, COUNT(DISTINCT c.customerNumber) AS n_customers
    FROM offices o
    LEFT JOIN employees e ON o.officeCode = e.officeCode
    LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY o.officeCode, o.city
    ORDER BY n_customers DESC;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_customers_per_office = get_customers_per_office()
#display the result
print(df_customers_per_office)

  officeCode           city  n_customers
0          4          Paris           29
1          7         London           17
2          3            NYC           15
3          1  San Francisco           12
4          2         Boston           12
5          6         Sydney           10
6          5          Tokyo            5


## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [49]:
# CodeGrade step10
# Replace None with your code
# List of employees with less than 20 customers, their office, and city 
def get_employees_with_few_customers():
    query = """
    SELECT e.employeeNumber, e.firstName, e.lastName, o.city, COUNT(c.customerNumber) AS n_customers
    FROM employees e
    JOIN offices o ON e.officeCode = o.officeCode
    LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY e.employeeNumber, e.firstName, e.lastName, o.city
    HAVING n_customers < 20
    ORDER BY n_customers ASC;
    """
    df = pd.read_sql(query, conn)
    return df
# Call the function to execute the query
df_employees_with_few_customers = get_employees_with_few_customers()
#display the result
print(df_employees_with_few_customers)

   employeeNumber firstName   lastName           city  n_customers
0            1002     Diane     Murphy  San Francisco            0
1            1056      Mary  Patterson  San Francisco            0
2            1076      Jeff   Firrelli  San Francisco            0
3            1088   William  Patterson         Sydney            0
4            1102    Gerard     Bondur          Paris            0
5            1143   Anthony        Bow  San Francisco            0
6            1619       Tom       King         Sydney            0
7            1625   Yoshimi       Kato          Tokyo            0
8            1611      Andy     Fixter         Sydney            5
9            1612     Peter      Marsh         Sydney            5
10           1621      Mami      Nishi          Tokyo            5
11           1165    Leslie   Jennings  San Francisco            6
12           1166    Leslie   Thompson  San Francisco            6
13           1188     Julie   Firrelli         Boston         

### Close the connection

In [50]:
# Run this cell without changes

conn.close()